In [2]:
# Import Libraries and Dependencies 

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import urllib.request
import matplotlib.pyplot as plt
import os
import json
from pathlib import Path
from datetime import datetime, timedelta
from tiingo import TiingoClient
import seaborn as sb
%matplotlib inline
import pandas_montecarlo
#pip install --user --upgrade matplotlib
#pip install pandas_montecarlo

ModuleNotFoundError: No module named 'tiingo'

In [ ]:
# grab data set from url *multipul updates each day make sure we are on current tab....

cv_data = pd.read_html('https://docs.google.com/spreadsheets/d/1wQVypefm946ch4XDp37uZ-wartW4V7ILdg-qYiDXUHM/htmlview?usp=sharing&sle=true#')

In [ ]:
#combine all tabs from data set
cv_data =pd.concat(cv_data)
cv_data

In [ ]:

#remove Strings from 1st row
cv_data.drop([0], inplace=True)
cv_data

In [ ]:
#drop "Unnamed:0"

#cv_data =cv_data.drop(columns=['Unnamed: 0','G','H','I'], inplace=True)

#cv_data

#rename columns in dataframe

cv_data = cv_data.rename(columns={
    'A':'Province/State',
    'B':'Country/Region',
    'C':'Last_Update',
    'D':'Infected',
    'E':'Deaths',
    'F':'Recovered',
    
})

In [ ]:
cv_data.head()

In [ ]:
#set index

cv_data =cv_data.set_index('Last_Update')

cv_data

In [ ]:
#drop "Unnamed:0"

cv_data =cv_data.drop(columns=['Unnamed: 0','G','H','I'], inplace=False)




In [ ]:
cv_data

In [ ]:
cv_data.isnull()

In [ ]:
cv_data.dropna(inplace=True)
cv_data

In [ ]:
cv_data.isnull().sum()
cv_data.head()

#cv_data['ratio']= cv_data['Deaths']/ cv_data['Confirmed']

cv_data.dtypes

In [ ]:
# change types from object to int.
#people_df['Person_ID'] = people_df['Person_ID'].astype('int')

cv_data['Infected'] = cv_data['Infected'].astype('int')
cv_data['Deaths'] = cv_data['Deaths'].astype('int')
cv_data['Recovered'] = cv_data['Recovered'].astype('int')

cv_data

In [ ]:
cv_data[['Province/State']].duplicated()

In [ ]:
#create a for loop to clean up duplicates for each day and get a running total for that days end.
#just use cumprod feature?
cv_data_days= cv_data.drop_duplicates(subset=['Province/State'], inplace=False)

In [ ]:
cv_data_days

In [ ]:
cv_data_days['Deaths'].sum()

In [ ]:
cv_data_days['Infected'].sum()

In [ ]:
cv_data_days['Recovered'].sum()

In [ ]:
#add totals of columns together for their individual daily total to be added to df daily
# replace index with date use current 

cv_data_2= cv_data.reset_index()

In [ ]:
cv_data_2


In [ ]:

#add new date column to get rid of time stamp
cv_data_2['Last_Update']= pd.to_datetime(cv_data_2.Last_Update)
cv_data_2['Date']= cv_data_2['Last_Update'].dt.strftime('%m/%d/%Y')
print (cv_data_2)


In [ ]:
# Set index as 'Date'
cv_data_2.set_index('Date')

In [ ]:
cv_data_2.drop(columns=['Last_Update'])

In [ ]:
#drop duplicates in order to get the last value for the day
cv_data_2.drop_duplicates(subset=['Province/State', "Date"], inplace=True)
cv_data_2

In [ ]:
#create new data frame extracting only mainland China info
cv_data_china=cv_data_2.loc[cv_data_2['Country/Region']=='Mainland China']
cv_data_china

In [ ]:
#Group by 'Date' in order to sum all
all_sum_cv = cv_data_china.groupby('Date').sum()

In [ ]:
all_sum_cv.reset_index(inplace=True)

In [ ]:
all_sum_cv.Date = pd.to_datetime(all_sum_cv.Date)

In [ ]:
all_sum_cv.set_index('Date',inplace=True) 

In [ ]:
#Find ratio of deaths to confirmed effected
#round decimals .000 
all_sum_cv['death_ratio']= all_sum_cv['Deaths']/ all_sum_cv['Infected']
all_sum_cv['recovered_ratio'] =all_sum_cv['Recovered']/ all_sum_cv['Infected']
#decimals =pd.Series ([3,3], index=['death_ratio', 'recovered_ratio'])
#cv_data.round(decimals)

In [ ]:
all_sum_cv['recovered/death'] =all_sum_cv['Recovered']/all_sum_cv['Deaths']
all_sum_cv.head()

In [ ]:
all_sum_cv.plot()

In [ ]:
##Monte Carlo Simulation on Corona Virus

In [ ]:
#jjp - fixed name
infection_change = all_sum_cv.pct_change()

In [ ]:
infection_change.head()

In [ ]:
# below is Stock Data

In [ ]:
# Bring Env Variables into Python/Store in Variable
tiingo = os.getenv('TIINGO_API_KEY')

In [ ]:
# Create a Client to Connect to Tiingo
client = TiingoClient()

In [ ]:
# Create Start and End Dates for 1 Year's Worth of Historical Data
end = datetime.now()
start = end + timedelta(-30)

In [ ]:
# Create DataFrames for China
china_ticker_df = client.get_dataframe(['ASHR','CQQQ', 'CHIE', 'CHIM', 'KURE', 'CHIS'],
                                            metric_name='close',
                                            startDate= start,
                                            endDate= end,
                                            frequency='daily')

In [ ]:
china_df = china_ticker_df.rename(columns={'ASHR':'ASHR (China SH Composite)','CQQQ':'CQQQ (China Technology)', 'CHIE':'CHIE (China Energy)', 'CHIM':'CHIM (China Materials)', 'KURE':'KURE (China Healthcare)', 'CHIS':'CHIS (China Consumer Staples)'})
china_df.head()

In [ ]:
# Count/Drop Nulls (if any)
china_df.isnull().sum()

In [ ]:
china_df.dropna(inplace=True)
china_df.isnull().sum()

In [ ]:
china_daily_returns = china_df.pct_change()
china_daily_returns.head()

In [ ]:
us_ticker_df = client.get_dataframe(['SPY','IGM', 'IYE', 'IYM', 'IYH', 'IECS'],
                                            metric_name='close',
                                            startDate= start,
                                            endDate= end,
                                            frequency='daily')

In [ ]:
us_df = us_ticker_df.rename(columns={'SPY':'SPY (USA SP500 Composite)', 'IGM':'IGM (USA Technology)', 'IYE':'IYE (USA Energy)', 'IYM':'IYM (USA Materials)', 'IYH':'IYH (USA Healthcare)', 'IECS':'IECS (USA Consumer Staples)'})

us_df.head()

In [ ]:
us_daily_returns = us_df.pct_change()
us_daily_returns.head()

In [ ]:
china_daily_returns.plot(title="Daily Returns for China ETFs")

In [ ]:
us_daily_returns.plot(title="Daily Returns for US ETFs")

In [ ]:
china_std = china_daily_returns.std()
china_std

In [ ]:
us_std = us_daily_returns.std()
us_std

In [ ]:
china_std.plot.bar()

In [ ]:
us_std.plot.bar()

In [ ]:
#only need to run this once to normalize the 
all_sum_cv.index = all_sum_cv.index.tz_localize('UTC')

In [ ]:
## Correlation Studies

In [ ]:
china_cv_raw_df = pd.concat([china_df, all_sum_cv], axis=1, sort=True)
china_cv_raw_df.dropna(inplace=True)

In [ ]:
usa_cv_raw_df = pd.concat([us_df, all_sum_cv], axis=1, sort=True)
usa_cv_raw_df.dropna(inplace=True)

In [ ]:
usa_china_raw_df = pd.concat([us_df, china_df], axis=1, sort=True)
usa_china_raw_df.dropna(inplace=True)

In [ ]:
all_data_raw_df = pd.concat([us_df, china_df, all_sum_cv], axis=1, sort=True)
all_data_raw_df.dropna(inplace=True)
#may need to take out duplicates

In [ ]:
# China and the Coronavirus Correlations
#help from: https://levelup.gitconnected.com/pearson-coefficient-of-correlation-using-pandas-ca68ce678c04
#help from https://datascience.stackexchange.com/questions/17540/make-seaborn-heatmap-bigger
china_correl = china_cv_raw_df.pct_change().corr(method='pearson')

fig = plt.gcf()
figsize = fig.get_size_inches()
fig.set_size_inches(figsize * 2)
sb.heatmap(china_correl, 
            xticklabels=china_correl.columns,
            yticklabels=china_correl.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)

In [ ]:
# USA and the Coronavirus Correlations
#help from: https://levelup.gitconnected.com/pearson-coefficient-of-correlation-using-pandas-ca68ce678c04
#help from https://datascience.stackexchange.com/questions/17540/make-seaborn-heatmap-bigger
usa_correl = usa_cv_raw_df.pct_change().corr(method='pearson')

fig = plt.gcf()
figsize = fig.get_size_inches()
fig.set_size_inches(figsize * 2)
sb.heatmap(usa_correl, 
            xticklabels=usa_correl.columns,
            yticklabels=usa_correl.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)

In [ ]:
# China and the USA Correlations
#help from: https://levelup.gitconnected.com/pearson-coefficient-of-correlation-using-pandas-ca68ce678c04
#help from https://datascience.stackexchange.com/questions/17540/make-seaborn-heatmap-bigger
usa_china_correl = usa_china_raw_df.pct_change().corr(method='pearson')
fig = plt.gcf()
figsize = fig.get_size_inches()
fig.set_size_inches(figsize * 2)
sb.heatmap(usa_china_correl, 
            xticklabels=usa_china_correl.columns,
            yticklabels=usa_china_correl.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)

In [ ]:
mc_corona = all_sum_cv['Infected'].montecarlo(sims=100, bust=-0.1, goal=1)

In [ ]:
mc_corona.plot()

In [ ]:
df_mc_corona =  mc_corona.data

In [ ]:
df_all_data_pct_chg = all_data_raw_df.pct_change()*100

In [ ]:
plt.hist(df_all_data_pct_chg['SPY (USA SP500 Composite)'])

In [ ]:
plt.hist(df_all_data_pct_chg['ASHR (China SH Composite)'])

In [ ]:
plt.hist(df_all_data_pct_chg['recovered_ratio'])

# Plot of China/ US composites over Coronavirus Infection rate

In [ ]:
#created a double y axis chart to show percentage changes of market versus coronovirus
#looked for help on: https://stackoverflow.com/questions/14762181/adding-a-y-axis-label-to-secondary-y-axis-in-matplotlib/14762601
import matplotlib.ticker as mtick

x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['ASHR (China SH Composite)']
y2 = df_all_data_pct_chg['SPY (USA SP500 Composite)']
y3 = df_all_data_pct_chg['Infected']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Daily Percent Change of Markets and Coronoavirus')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['ASHR China SH Composite', 'SPY (USA SP500 Composite)', 'Infected'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Energy sectors over Daily infection rate of Coronavirus

In [ ]:
#created a double y axis chart to show percentage changes of market versus coronovirus
#looked for help on: https://stackoverflow.com/questions/14762181/adding-a-y-axis-label-to-secondary-y-axis-in-matplotlib/14762601
import matplotlib.ticker as mtick

x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['CHIE (China Energy)']
y2 = df_all_data_pct_chg['IYE (USA Energy)']
y3 = df_all_data_pct_chg['Infected']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Daily Percent Change of Markets and Coronoavirus')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['CHIE (China Energy)', 'IYE (USA Energy)', 'Infected'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Energy sectors over recovery rate of Corona Virus

In [ ]:
#created a double y axis chart to show percentage changes of market versus coronovirus
#looked for help on: https://stackoverflow.com/questions/14762181/adding-a-y-axis-label-to-secondary-y-axis-in-matplotlib/14762601
import matplotlib.ticker as mtick

x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['CHIE (China Energy)']
y2 = df_all_data_pct_chg['IYE (USA Energy)']
y3 = df_all_data_pct_chg['recovered_ratio']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Daily Percent Change of Markets and Coronoavirus')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-100, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['CHIE (China Energy)', 'IYE (USA Energy)', 'recovered_ratio'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Tech markets over infection rate of Corona Virus

In [ ]:
x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['CQQQ (China Technology)']
y2 = df_all_data_pct_chg['IGM (USA Technology)']
y3 = df_all_data_pct_chg['Infected']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Tech Markets VS Infection Rate')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['CQQQ China Tech.', 'IGM USA Tech.', 'Infected'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Tech markets over recovery rate of Corona Virus

In [ ]:
x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['CQQQ (China Technology)']
y2 = df_all_data_pct_chg['IGM (USA Technology)']
y3 = df_all_data_pct_chg['recovered_ratio']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Tech Markets VS Recovery Ratio')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['CQQQ China Tech.', 'IGM USA Tech.', 'Recovery Ratio'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Materials Markets VS Infection Rate

In [ ]:
x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['CHIM (China Materials)']
y2 = df_all_data_pct_chg['ITM (USA Materials)']
y3 = df_all_data_pct_chg['Infected']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('MAterials Market VS Infection Rate')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['CHIM China Materials', 'IYM USA Materials', 'Infected'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Materials Markets Vs Recovery Rate of Corona Virus

In [ ]:
x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['CHIM (China Materials)']
y2 = df_all_data_pct_chg['IYM (USA Materials)']
y3 = df_all_data_pct_chg['recovered_ratio']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Materials Market Vs Recovery Rate')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['CHIM CHina Materials', 'IYM USA Materials', 'Recovered'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Healthcare Markets Vs Infection Rate 

In [ ]:
x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['KURE (China Healthcare)']
y2 = df_all_data_pct_chg['IYH (USA Healthcare)']
y3 = df_all_data_pct_chg['Infected']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Healthcare Markets Vs Infection rate')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['KURE China Healthecare', 'IYH USA Healthcare', 'Infected'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Healthcare Markets Vs Recovery Rate

In [ ]:
x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['KURE (China Healthecare)']
y2 = df_all_data_pct_chg['IYH (USA Healthcare)']
y3 = df_all_data_pct_chg['recovered_ratio']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Healthcare Markets Vs Recovery Rate')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['KURE China Healthcare', 'IYH USA Healthcare', 'Recovered'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Consumer Staples Markets Vs Infection Rate

In [ ]:
x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['CHIS (China Consumer Staples']
y2 = df_all_data_pct_chg['IECS (USA Consumer Staples)']
y3 = df_all_data_pct_chg['Infected']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Consumer Staples Vs Infection rate')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['CHIS China Consumer Satples', 'IECS USA Consumer Staples', 'Infected'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()

# Plot of Consumer Staples Markets Vs Recovery Rate

In [ ]:
x = df_all_data_pct_chg.index
y1 = df_all_data_pct_chg['CHIS (China Consumer Staples)']
y2 = df_all_data_pct_chg['IECS (USA Consumer Staples)']
y3 = df_all_data_pct_chg['recovered_ratio']

fig, ax1 = plt.subplots(figsize=(10,5))

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-', y2, 'b-')
ax2.plot(x, y3, 'r-')
                   
ax1.set_title('Consumer Staples Vs. Recovery Ratio')
ax1.set_xlabel('Trade Days')
ax1.set_ylabel('Trade Index Daily Percent Change', color='g')
ax2.set_ylabel('Coronoavirus Daily Percent Change', color='r')

ax2.set_ylim(-25, 175)

fig.autofmt_xdate()

ax1.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

#looked up how to create custom legend: https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html
#looked up how to lock legend out of chart: https://pythonspot.com/matplotlib-legend/
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['CHIS China Consumer Staples', 'IECS USA Consumer Staples', 'Recovered'], loc='upper center', bbox_to_anchor=(0.5, -.33),shadow=True, ncol=2)

plt.show()